In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import data
from skimage.util import img_as_float
from skimage.filters import gabor_kernel

In [5]:
def getPath_and_dict(path, train):
    lichens = os.listdir(path)
    dic_lic = {}
    res = []
    for i,l in enumerate(lichens):
        dic_lic[l] = i
        pth = os.path.join(path,l)
        images_of_specific_lichen = os.listdir(pth)
        for j,im in enumerate(images_of_specific_lichen):
            image = os.path.join(pth,im)
            res.append(image)
    
    
    if(train is True):
        np.random.shuffle(res)
    return res, dic_lic


def readimage(im):
    img = cv2.imread(im)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img



In [6]:
# prepare filter bank kernels--> frequenza + inversamente proporzionale alla sigma 
def create_kernels(orientations, frequency_vec):
    kernels = []
    for theta in range(orientations):
        theta = theta / 4. * np.pi

        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta,sigma_x=None, sigma_y=None))
            kernels.append(kernel)
    return kernels 


In [7]:
def compute_feats(image, kernels):
    feats_r = np.zeros((len(kernels), 2), dtype=np.double)
    feats_g = np.zeros((len(kernels), 2), dtype=np.double)
    feats_b = np.zeros((len(kernels), 2), dtype=np.double)
    
    img_r = image[:,:,0]
    img_g = image[:,:,1]
    img_b = image[:,:,2]
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(img_r, kernel, mode='wrap')
        feats_r[k, 0] = filtered.mean()
        feats_r[k, 1] = filtered.var()

        filtered = ndi.convolve(img_g, kernel, mode='wrap')
        feats_g[k, 0] = filtered.mean()
        feats_g[k, 1] = filtered.var()

        filtered = ndi.convolve(img_b, kernel, mode='wrap')
        feats_b[k, 0] = filtered.mean()
        feats_b[k, 1] = filtered.var()
    
    feats_r = feats_r.reshape(-1)
    feats_g = feats_g.reshape(-1)
    feats_b = feats_b.reshape(-1)
    
    feats = np.concatenate([feats_r, feats_g, feats_b])
    feats = feats.reshape(-1)
    return feats


In [8]:
def obtain_descriptors(path, orientations, frequency_vec,train = True):
    
    kernels = create_kernels(orientations, frequency_vec)
    images_pth, label_dict = getPath_and_dict(path, train)
    descriptor_list = []    
    train_labels = []
    image_count = len(images_pth) 
    for i,img_path in enumerate(images_pth):
        print('----')
        print(i)
        print('----')
        cl = img_path.split('/')[-2]
        class_index = cl
        train_labels.append(class_index)        
        img = readimage(img_path)
        dsc = compute_feats(img, kernels)
        if(i%100==0):
            print(dsc.shape)
        descriptor_list.append(dsc)     
    descriptors =np.array(descriptor_list)
    train_labels = np.array(train_labels)
    return descriptors, train_labels

In [ ]:
orient = 3
frequency_vec = [0.25/(np.sqrt(2)**j) for j in range(4)]
import os
import cv2

In [ ]:
train_desc, train_lab = obtain_descriptors('../../../data/train',orient, frequency_vec)

In [ ]:
c.shape

In [ ]:
from scipy.io import loadmat
def load_descriptor_from_matfile(ft_path,lab_path):
    feat = loadmat(ft_path)
    feat = feat['dsc']
    lab = loadmat(lab_path)
    lab = lab['lab']
    return feat, lab


training_feat,tr_lab = load_descriptor_from_matfile('dsc/training_descriptors.mat', 'dsc/training_labels.mat')
tst_features, tst_lab = load_descriptor_from_matfile('dsc/test_descriptors.mat','dsc/test_labels.mat')
tr_lab = tr_lab -1
tst_lab = tst_lab -1

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# import the necessary packages
def search_for_best_rbf(training_feat, tr_lab, tst_features, tst_lab):
    param_grid = {'C': [ 1e0, 1e1, 1e2,1e3], 'gamma': [   0.01, 0.1,1]}
    clf0 = GridSearchCV(SVC(kernel='rbf'), param_grid)
    svc = clf0.fit(training_feat, tr_lab)
    print("Best estimator found by grid search : ", clf0.best_estimator_)
    y_pred = clf0.predict(tst_features)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, tst_lab))
    return clf0,skmetrics.accuracy_score(y_pred, tst_lab)

In [ ]:
from sklearn.svm import SVC
import sklearn.metrics as skmetrics
a, b = search_for_best_rbf(training_feat, tr_lab, tst_features, tst_lab)